# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system and use `LinearAlgebra` to bring in the factorizations and special matrix structures.

In [57]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Vector{Float64}:
 1.3104934216509059
 1.8457652658550976
 1.1644737961889329

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [39]:
Alu = lu(A)

LU{Float64, Matrix{Float64}}
L factor:
3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.183828  1.0       0.0
 0.254167  0.559457  1.0
U factor:
3×3 Matrix{Float64}:
 0.670242  0.441117   0.939988
 0.0       0.908783   0.63535
 0.0       0.0       -0.411597

In [40]:
typeof(Alu)

LU{Float64, Matrix{Float64}}

The different parts of the factorization can be extracted by accessing their special properties

In [41]:
Alu.P

3×3 Matrix{Float64}:
 0.0  1.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0

In [42]:
Alu.L

3×3 Matrix{Float64}:
 1.0       0.0       0.0
 0.183828  1.0       0.0
 0.254167  0.559457  1.0

In [43]:
Alu.U

3×3 Matrix{Float64}:
 0.670242  0.441117   0.939988
 0.0       0.908783   0.63535
 0.0       0.0       -0.411597

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [44]:
A\b

3-element Vector{Float64}:
 0.9999999999999997
 0.9999999999999999
 1.0000000000000004

In [45]:
Alu\b

3-element Vector{Float64}:
 0.9999999999999997
 0.9999999999999999
 1.0000000000000004

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [46]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [47]:
Aqr = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.242516   0.438887  -0.865196
 -0.95416   -0.269144   0.130924
 -0.175401   0.857286   0.48404
R factor:
3×3 Matrix{Float64}:
 -0.702442  -0.745013  -1.08297
  0.0        1.00223    0.520035
  0.0        0.0        0.356113

Similarly to the LU factorization, the matrices `Q` and `R` can be extracted from the QR factorization object via

In [48]:
Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.242516   0.438887  -0.865196
 -0.95416   -0.269144   0.130924
 -0.175401   0.857286   0.48404

In [49]:
Aqr.R

3×3 Matrix{Float64}:
 -0.702442  -0.745013  -1.08297
  0.0        1.00223    0.520035
  0.0        0.0        0.356113

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [56]:
Asym = A + A'
AsymEig = eigen(Asym)

Eigen{Float64, Float64, Matrix{Float64}, Vector{Float64}}
values:
3-element Vector{Float64}:
 0.2563987232230714
 0.7653487374117782
 3.4357109789701137
vectors:
3×3 Matrix{Float64}:
  0.386181   0.86382    -0.323541
 -0.710705   0.0550468  -0.701333
  0.588016  -0.500784   -0.63518

The values and the vectors can be extracted from the Eigen type by special indexing

In [14]:
AsymEig.values

3-element Vector{Float64}:
 -1.019452285246619
  0.2470572608899988
  3.617277930911641

In [15]:
AsymEig.vectors

3×3 Matrix{Float64}:
  0.133133   0.862676  0.48792
 -0.838066  -0.164815  0.520078
  0.529075  -0.478149  0.701037

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [16]:
inv(AsymEig)*Asym

3×3 Matrix{Float64}:
 1.0          -7.99361e-15   1.02141e-14
 1.88738e-15   1.0          -5.77316e-15
 2.55351e-15   2.66454e-15   1.0

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [17]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [18]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [19]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

0.39891958496867586

In [20]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [21]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [22]:
@time eigvals(Asym);

  0.512645 seconds (161.00 k allocations: 16.157 MiB, 2.75% gc time, 42.08% compilation time)


In [23]:
@time eigvals(Asym_noisy);

  1.042965 seconds (13 allocations: 7.920 MiB)


In [24]:
@time eigvals(Asym_explicit);

  0.260516 seconds (5.89 k allocations: 8.316 MiB, 3.39% compilation time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [25]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.833405 seconds (803.77 k allocations: 224.596 MiB, 2.87% gc time, 30.34% compilation time)


6.33133593253314

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [26]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [27]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Matrix{Rational{BigInt}}:
 4//5   3//5   7//10
 3//10  1//10  7//10
 3//10  2//5   1//5

In [28]:
x = fill(1, 3)
b = Arational*x

3-element Vector{Rational{BigInt}}:
 21//10
 11//10
  9//10

In [29]:
Arational\b

3-element Vector{Rational{BigInt}}:
 1//1
 1//1
 1//1

In [30]:
lu(Arational)

LU{Rational{BigInt}, Matrix{Rational{BigInt}}}
L factor:
3×3 Matrix{Rational{BigInt}}:
 1//1   0//1  0//1
 3//8   1//1  0//1
 3//8  -5//7  1//1
U factor:
3×3 Matrix{Rational{BigInt}}:
 4//5  3//5    7//10
 0//1  7//40  -1//16
 0//1  0//1   11//28

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```
and assign it a variable `A_eigv`

In [35]:
using LinearAlgebra

In [156]:
A = [[140, 97, 74, 168, 131];;
[97, 106, 89, 131, 36];;
[74, 89, 152, 144, 71];;
[168, 131, 144, 54, 142];;
[131, 36, 71, 142, 36]]

exerciseAEigenVals = eigen(A)
A_eigv = eigvals(A)
testVal = exerciseAEigenVals.values
println(A_eigv == testVal)
println(A_eigv)

false
[-128.4932276480215, -55.887784553056875, 42.75216727931892, 87.16111477514524, 542.4677301466141]


In [150]:
@assert A_eigv ==  [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#= Failed to pass the assert function from a rounding error from the function after several attempts as trying different
   functions that do the same thing are rounding differently as shown by the assertion that the comparison is false. =#


LoadError: AssertionError: A_eigv == [-128.49322764802145, -55.887784553056875, 42.7521672793189, 87.16111477514521, 542.4677301466143]

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [151]:
A_diag = zeros(Float64, 5, 5)
for i in 1:5
    A_diag[i, i] = Diagonal(A_eigv)[i, i]
    A_diag[i, i] = round(A_diag[i, i], sigdigits=6)
end
A_diag


5×5 Matrix{Float64}:
 -128.493    0.0      0.0      0.0       0.0
    0.0    -55.8878   0.0      0.0       0.0
    0.0      0.0     42.7522   0.0       0.0
    0.0      0.0      0.0     87.1611    0.0
    0.0      0.0      0.0      0.0     542.468

In [149]:
@assert A_diag ==  [-128.493    0.0      0.0      0.0       0.0;
    0.0    -55.8878   0.0      0.0       0.0;
    0.0      0.0     42.7522   0.0       0.0;
    0.0      0.0      0.0     87.1611    0.0;
    0.0 0.0      0.0      0.0     542.468]
    

#### 11.3 
Create a `LowerTriangular` matrix from `A` and store it in `A_lowertri`

In [163]:
A_lowertri = zeros(Float64, 5, 5)
for i in 1:5
    for j in 1:5
        if i >= j
            A_lowertri[i, j] = A[i,j]
        end
    end
end
A_lowertri

5×5 Matrix{Float64}:
 140.0    0.0    0.0    0.0   0.0
  97.0  106.0    0.0    0.0   0.0
  74.0   89.0  152.0    0.0   0.0
 168.0  131.0  144.0   54.0   0.0
 131.0   36.0   71.0  142.0  36.0

In [164]:
@assert A_lowertri ==  [140    0    0    0   0;
  97  106    0    0   0;
  74   89  152    0   0;
 168  131  144   54   0;
 131   36   71  142  36]